In [2]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [3]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_full_agriVision.ipynb


In [4]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [5]:
from scipy import spatial
from sklearn.decomposition import PCA

In [6]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [7]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [8]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [9]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [15]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,436364.91,-0.19,-0.23,-1.39,0.09,0.05,0.05,-0.03,-0.00
3,-0.19,75656.18,3.66,-0.10,0.31,-0.01,0.01,0.00,0.00
4,-0.23,3.66,14669.75,0.02,0.12,-0.01,0.00,-0.00,0.00
5,-1.39,-0.10,0.02,2082.43,0.03,0.02,-0.01,-0.00,-0.00
6,0.09,0.31,0.12,0.03,475.80,0.00,-0.00,0.00,-0.00
7,0.05,-0.01,-0.01,0.02,0.00,108.68,0.00,-0.00,-0.00
8,0.05,0.01,0.00,-0.01,-0.00,0.00,32.88,-0.00,0.00
9,-0.03,0.00,-0.00,-0.00,0.00,-0.00,-0.00,3.64,-0.00
10,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.18


In [16]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00000,-0.00000,-0.00005,0.00001,0.00001,0.00001,-0.00002,-0.00001
3,-0.00000,1.00000,0.00011,-0.00001,0.00005,-0.00000,0.00001,0.00000,0.00001
4,-0.00000,0.00011,1.00000,0.00000,0.00004,-0.00001,0.00000,-0.00001,0.00001
5,-0.00005,-0.00001,0.00000,1.00000,0.00003,0.00005,-0.00003,-0.00003,-0.00002
6,0.00001,0.00005,0.00004,0.00003,1.00000,0.00002,-0.00003,0.00000,-0.00002
7,0.00001,-0.00000,-0.00001,0.00005,0.00002,1.00000,0.00002,-0.00001,-0.00002
8,0.00001,0.00001,0.00000,-0.00003,-0.00003,0.00002,1.00000,-0.00003,0.00000
9,-0.00002,0.00000,-0.00001,-0.00003,0.00000,-0.00001,-0.00003,1.00000,-0.00004
10,-0.00001,0.00001,0.00001,-0.00002,-0.00002,-0.00002,0.00000,-0.00004,1.00000


In [17]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

5.579607582710743

In [13]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.11677892e+10 6.26245044e+08 2.30527550e+07 4.52497843e+05
 2.26734637e+04 1.11607626e+03 8.93488067e+01 8.26259506e-01
 4.38065019e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,9.997418e-01,-0.022325,-0.004189,-0.000597,-0.000136,-0.000031,-0.000009,-0.000001,-6.043942e-08
1,2.219271e-02,0.999331,-0.028787,-0.003924,-0.000892,-0.000205,-0.000062,-0.000007,-3.160449e-07
2,4.810134e-03,0.028579,0.999270,-0.024244,-0.005408,-0.001237,-0.000374,-0.000042,-1.990790e-06
3,7.905331e-04,0.004544,0.023817,0.998487,-0.048146,-0.010337,-0.003125,-0.000347,-1.715644e-05
4,2.160105e-04,0.001242,0.006423,0.047230,0.996884,-0.060409,-0.017200,-0.001890,-9.358428e-05
5,6.024390e-05,0.000348,0.001795,0.012654,0.057584,0.992090,-0.110280,-0.010741,-5.299455e-04
6,2.520509e-05,0.000146,0.000753,0.005311,0.023313,0.108245,0.992178,-0.057327,-2.775951e-03
7,4.211920e-06,0.000024,0.000124,0.000873,0.003815,0.016711,0.055554,0.997078,-4.955402e-02
8,4.219233e-07,0.000002,0.000012,0.000086,0.000377,0.001651,0.005454,0.049305,9.987674e-01


In [14]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.0002582 , 0.00066896, 0.00072956, 0.00151279, 0.00311591,
       0.00790957, 0.00782227, 0.00292248, 0.00123256])